In [66]:
import pandas as pd

from arcgis import GIS
from arcgis.mapping import * #WebMap, MapServiceLayer, MapFeatureLayer

gis = GIS('home')

### Flood Prone Areas
* https://www.fema.gov/flood-maps/national-flood-hazard-layer
* https://www.coast.noaa.gov/arcgis/rest/services/FloodExposureMapper/CFEM_FEMAFloodZones/MapServer/1
* https://landscape11.arcgis.com/arcgis/rest/services/USA_Flood_Hazard_Areas/ImageServer
* https://dukeuniv.maps.arcgis.com/home/item.html?id=3a2a84ccaa824fb6a87087553bf25f92


In [47]:
fema_web_map_app = gis.content.get('8b0adb51996444d4879338b5529aa9cd')
fema_web_map_app_data = fema_web_map_app.get_data()
fema_web_map_app_ID = fema_web_map_app_data['map']['itemId']
fema_web_map_item = gis.content.get(fema_web_map_app_ID)
fema_web_map = WebMap(fema_web_map_item)
fema_web_map_layers = fema_web_map.layers

In [53]:
#Show the layer titles
[layer['title'] for layer in fema_web_map_layers]

['Effective FIRM Panels',
 'NFHL',
 'Coastal Barrier Resources System Area (US FWS)']

In [59]:
#Create a map service layer
fema_ms_layer = MapServiceLayer(fema_web_map_layers[1]['url'])
fema_layers = fema_ms_layer.properties['layers']
len(fema_layers)

32

In [63]:
df_layers = pd.DataFrame(fema_layers)
df_layers.loc[0]

id                                          0
name                        NFHL Availability
parentLayerId                              -1
defaultVisibility                       False
subLayerIds                              None
minScale                                  0.0
maxScale                                72225
type                            Feature Layer
geometryType              esriGeometryPolygon
supportsDynamicLegends                   True
Name: 0, dtype: object

In [67]:
ms = MapTable(lyrs[1]['url'])
ms.properties['layers']

NameError: name 'MapTable' is not defined

In [65]:
mf_lyr = MapFeatureLayer(lyrs[1]['url'])
mf_lyr

<MapFeatureLayer url:"https://hazards.fema.gov/arcgis/rest/services/FIRMette/NFHLREST_FIRMette/MapServer">

### Sea Level Rise
* https://www.coast.noaa.gov/arcgis/rest/services/dc_slr/slr_1ft/MapServer/1
* https://www.coast.noaa.gov/arcgis/rest/services/dc_slr/slr_1ft/MapServer/0
* https://www.coast.noaa.gov/arcgis/rest/services/dc_slr/slr_2ft/MapServer/1
* https://www.coast.noaa.gov/arcgis/rest/services/dc_slr/slr_2ft/MapServer/0
* https://www.coast.noaa.gov/arcgis/rest/services/dc_slr/slr_3ft/MapServer/1
* https://www.coast.noaa.gov/arcgis/rest/services/dc_slr/slr_3ft/MapServer/0
* https://coast.noaa.gov/slrdata/DEMs/NC/index.html

### Storm Surge
* https://tiles.arcgis.com/tiles/C8EMgrsFcRFL6LrL/arcgis/rest/services/Storm_Surge_HazardMaps_Category1_v3/MapServer?f=html&cacheKey=8c1eda5e290c30bd
* https://tiles.arcgis.com/tiles/C8EMgrsFcRFL6LrL/arcgis/rest/services/Storm_Surge_HazardMaps_Category2_v3/MapServer?f=html&cacheKey=bc29bd820e1e09aa
* https://tiles.arcgis.com/tiles/C8EMgrsFcRFL6LrL/arcgis/rest/services/Storm_Surge_HazardMaps_Category3_v3/MapServer?f=html&cacheKey=990f926f2812a1e0
* https://tiles.arcgis.com/tiles/C8EMgrsFcRFL6LrL/arcgis/rest/services/Storm_Surge_HazardMaps_Category4_v3/MapServer?f=html&cacheKey=ba281138bc28168f
* https://tiles.arcgis.com/tiles/C8EMgrsFcRFL6LrL/arcgis/rest/services/Storm_Surge_HazardMaps_Category5_v3/MapServer?f=html&cacheKey=a407a7ec582a37b0

### Areas of low slope
* 

### Soil erodibility
* https://landscape11.arcgis.com/arcgis/rest/services/USA_Soils_Erodibility_Factor/ImageServer
* https://gdg.sc.egov.usda.gov/GDGHome_DirectDownLoad.aspx

### Impervious Surfaces
* https://coast.noaa.gov/arcgis/rest/services/Landcover/CCAP_High_Resolution_V2_Impervious/ImageServer
* https://coast.noaa.gov/arcgis/rest/services/Landcover/CCAP_High_Resolution_V2_Landcover/ImageServer
* https://coastalimagery.blob.core.windows.net/ccap-landcover/CCAP_bulk_download/High_Resolution_Land_Cover/Phase_1_Initial_Layers/index.html